In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import re

In [2]:
df = pd.read_csv('../data/full_data_2011-01-01_2022-11-26.csv', sep=',')
df_precipitation = pd.read_csv('../data/data_precipitation+2weeks.csv')
df_temperature = pd.read_csv('../data/temperature_data+2weeks.csv')

df = df.loc[[2]].drop(columns='Unnamed: 0').T
df.columns = ['energie']
df['The_date'] = df['energie']

ParserError: Error tokenizing data. C error: Expected 1 fields in line 1785, saw 2


In [3]:
dict_month = {'Jan': '01', 'Feb': '02', 'Mar': '03', 'Apr': '04', 'May': '05', 'Jun': '06', 'Jul': '07', 'Aug': '08','Sep': '09','Oct': '10','Nov': '11','Dec': '12'}
for i in range(len(df)):
    day = "".join(re.findall("\d", df.index[i].split(" ", 1)[0]))
    month = dict_month[df.index[i].split(" ")[1]]
    year = "".join(re.findall("\d", df.index[i].split(" ", 1)[1]))
    the_date = f'{day}-{month}-{year}'
    df['The_date'].iloc[i] = the_date
df['The_date'] = pd.to_datetime(df['The_date'])

In [4]:
df_rain = pd.read_csv('../data/data_precipitation.csv', sep=';')
df_temp= pd.read_csv('../data/temperature_data.csv', sep=';')
df_temp.drop(columns=['Unnamed: 17'], inplace = True)
df = df[2557:-18]
df.reset_index(inplace = True)

In [5]:
df_rain['mean'] = df_rain.mean(axis=1)
df_temp['mean'] = df_temp.mean(axis=1)


df['rain'] = df_rain['mean']
df['temp'] = df_temp['mean']
df

/var/folders/9g/tq1twg9d6mq7h7qdcrf8krk40000gn/T/ipykernel_10538/877483587.py:1: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  df_rain['mean'] = df_rain.mean(axis=1)
/var/folders/9g/tq1twg9d6mq7h7qdcrf8krk40000gn/T/ipykernel_10538/877483587.py:2: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  df_temp['mean'] = df_temp.mean(axis=1)


,index,energie,The_date,rain,temp
0,01 Jan 18 Daily peak [MW],944.0,2018-01-01,0.068182,3.718750
1,02 Jan 18 Daily peak [MW],2496.0,2018-02-01,0.409091,4.812500
2,03 Jan 18 Daily peak [MW],2576.0,2018-03-01,0.190909,5.890625
3,04 Jan 18 Daily peak [MW],1972.0,2018-04-01,0.109091,5.609375
4,05 Jan 18 Daily peak [MW],3649.0,2018-05-01,1.500000,4.468750
...,...,...,...,...,...
1769,05 Nov 22 Daily peak [MW],3445.0,2022-05-11,0.000000,10.593750
1770,06 Nov 22 Daily peak [MW],1901.0,2022-06-11,0.131818,11.578125
1771,07 Nov 22 Daily peak [MW],2631.0,2022-07-11,0.036364,12.250000
1772,08 Nov 22 Daily peak [MW],3023.0,2022-08-11,2.759091,13.390625


In [6]:
df['energie_ma'] = df['energie'].rolling(14).mean()
df['rain_ma'] = df['rain'].rolling(14).mean()
df['temp_ma'] = df['temp'].rolling(14).mean()

In [7]:
from tensorflow.keras.preprocessing import timeseries_dataset_from_array
from sklearn.preprocessing import StandardScaler

In [8]:
df_to_scale = ['energie_ma','rain_ma', 'temp_ma']
scaler = StandardScaler()

for column in df_to_scale:
            df[column] = scaler.fit_transform(pd.DataFrame(df[column],columns=[column]))

In [46]:
X=[]
x=[]
y = []
z = []
Z = []
for i in range(14,1300):
    x = []
    x.append(df.loc[i, 'temp_ma'])
    x.append(df.loc[i, 'rain_ma'])
    x.append(df.loc[i, 'energie_ma'])
    X.append(x)
    
for i in range(1300,1500):
    y.append(df.loc[i, 'energie_ma'])

X_train = np.array(X).astype(np.float32)
y_train = np.array(y).astype(np.float32) 

for i in range(1500,len(df)):
    z = []
    z.append(df.loc[i, 'temp_ma'])
    z.append(df.loc[i, 'rain_ma'])
    z.append(df.loc[i, 'energie_ma'])
    Z.append(x)

X_test = np.array([Z]).astype(np.float32)

In [48]:
for batch in dataset_train.take(1):
    inputs, targets = batch
print("Input shape:", inputs.numpy().shape)
print("Target shape:", targets.numpy().shape)

Input shape: (32, 150, 3)
Target shape: (32,)


In [50]:
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers

model = Sequential()
model.add(layers.LSTM(units=32, input_shape=(1286,3)))
model.add(layers.Dense(1, activation="linear"))

model.compile(loss='mse', 
              optimizer='adam')

model.fit(dataset_train,
         epochs=200, verbose=0)

2022-12-05 12:53:18.099698: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2022-12-05 12:53:18.181143: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2022-12-05 12:53:18.395546: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


In [51]:
model.summary()

Model: "sequential_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_8 (LSTM)               (None, 32)                4608      
                                                                 
 dense_8 (Dense)             (None, 1)                 33        
                                                                 
Total params: 4,641
Trainable params: 4,641
Non-trainable params: 0
_________________________________________________________________


In [52]:
X_test.shape

(1, 274, 3)

In [53]:
res = model.predict(X_test)
plt.plot(df.loc[1500:]['energie'])
plt.plot(res)

ValueError: in user code:

    File "/Users/alexander_wynaendts/.pyenv/versions/3.10.6/envs/lewagon/lib/python3.10/site-packages/keras/engine/training.py", line 2041, in predict_function  *
        return step_function(self, iterator)
    File "/Users/alexander_wynaendts/.pyenv/versions/3.10.6/envs/lewagon/lib/python3.10/site-packages/keras/engine/training.py", line 2027, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/Users/alexander_wynaendts/.pyenv/versions/3.10.6/envs/lewagon/lib/python3.10/site-packages/keras/engine/training.py", line 2015, in run_step  **
        outputs = model.predict_step(data)
    File "/Users/alexander_wynaendts/.pyenv/versions/3.10.6/envs/lewagon/lib/python3.10/site-packages/keras/engine/training.py", line 1983, in predict_step
        return self(x, training=False)
    File "/Users/alexander_wynaendts/.pyenv/versions/3.10.6/envs/lewagon/lib/python3.10/site-packages/keras/utils/traceback_utils.py", lin